# HB1 Visa

Dataset taken from https://www.kaggle.com/nsharan/h-1b-visa/version/2

In [2]:
#  Replace with your container and storage account:  "wasbs://<container>@<storage account>.blob.core.windows.net/"
pathPrefix = "wasbs://hb1-visa@vpldb.blob.core.windows.net/"
path = pathPrefix + "h1b_kaggle.csv"
# Load CSV
df = spark.read.option("header","true").csv(path)

In [3]:
# Manipulate a data-row to change some columns
def reworkRow(row):
  from pyspark.sql import Row

  # Let's convert the data row to a dictionary
  # This is easier to manipulate as the dictionary isn't readonly
  dict = row.asDict()
  
  # Change the first column from _co to id
  dict['id'] = dict['_c0']
  del(dict['_c0'])

  # Split the WORKSITE column into city & state
  worksite = dict['WORKSITE'].split(',')
  city = worksite[0].strip() if len(worksite)>0 else None
  state = worksite[1].strip() if len(worksite)>1 else None
  dict['CITY'] = city
  dict['STATE'] = state
  del(dict['WORKSITE'])
  
  # Change FULL_TIME _POSITION column from 'Y' and 'N' to True / False (boolean)
  dict['FULL_TIME_POSITION'] = True if dict['FULL_TIME_POSITION']=='Y' else False
  
  return Row(**dict)

In [4]:
# Convert back to RDD to manipulate the rows
rdd = df.rdd.map(lambda row: reworkRow(row))
# Create a dataframe with the manipulated rows
hb1 = spark.createDataFrame(rdd)
# Let's cache this bad boy
hb1.cache()
# Create a temporary view from the data frame
hb1.createOrReplaceTempView("hb1")

In [5]:
%sql
SELECT COUNT(*)
FROM hb1

In [6]:
%sql
SELECT STATE, COUNT(*) AS petitions
FROM hb1
GROUP BY STATE
ORDER BY petitions DESC

In [7]:
%sql
SELECT CITY, STATE, COUNT(*) AS petitions
FROM hb1
GROUP BY CITY, STATE
ORDER BY petitions DESC

In [8]:
%sql
SELECT CASE_STATUS, COUNT(*) AS petitions
FROM hb1
GROUP BY CASE_STATUS
ORDER BY petitions DESC

In [9]:
%sql
SELECT EMPLOYER_NAME, COUNT(*) as count
FROM hb1
GROUP BY EMPLOYER_NAME
ORDER BY count DESC
LIMIT 20

In [10]:
%sql
SELECT JOB_TITLE, COUNT(*) AS count
FROM hb1
GROUP BY JOB_TITLE
ORDER BY count DESC
LIMIT 20